### 前期准备工作
导入需要的库

In [7]:
import cv2
import os

准备好所有需要的文件和文件夹
1.  opencv 训练模型文件检查

In [8]:
def check_exe():
    files=os.listdir(os.getcwd())
    files1="opencv_createsamples"
    files2="opencv_traincascade"
    for i in files:
        if files1=="true" and files2=="true":
            break
        if i==files1:
            files1="true"
        elif i==files2:
            files2="true"
    if files1 != "true" or files2 != "true":
        print("缺失一些必要的训练文件，可以选择在网址\
        https://link.jscdn.cn/sharepoint/aHR0cHM6Ly\
            9tYWlsc2N1dGVkdWNuLW15LnNoYXJlcG9pbnQuY29\
                tLzp1Oi9nL3BlcnNvbmFsLzIwMTk2NjE3NjI1M\
                    19tYWlsX3NjdXRfZWR1X2NuL0VkQ2FLVVNL\
                        R2JwQ3FuNHpPbU5RN0M4QlFRbW5OQkZX\
                            TnJJYXN6MnRIRXRxUnc_ZT1hbzhrZW\
                                4.zip 中进行下载")
    else:
        print("done,1")
check_exe()

done,1


2.  接下来，创建一些必要的文件夹和文档来存放必要的照片和数据吧（这步操作会删除已有的很多东西，如果你确定你已经准备好的所有文件，请跳过这步）

In [9]:
neg_txt=open("neg.txt",'w+')
neg_txt.close()
pos_txt=open("pos.txt",'w+')
pos_txt.close()
def mkdir1():
    neg_dir='neg'
    pos_dir='pos'
    Cascade='TrainCascadeClassification'
    if os.path.exists(neg_dir):
    #先判断该文件夹是否已经存在， 
        if os.path.getsize(neg_dir):
            #如果存在，判断该文件夹里面是否有东西
            for file in os.listdir(neg_dir)[0:]:
                #删除该文件夹里面的东西
                #先通过os.listdir获取该文件夹内的文件名列表，然后逐一删除
                file_path=neg_dir+'/'+file 
                os.remove(file_path) #删除该文件
    else: 
        os.mkdir(neg_dir)
    if os.path.exists(pos_dir):
    #先判断该文件夹是否已经存在， 
        if os.path.getsize(pos_dir):
            #如果存在，判断该文件夹里面是否有东西
            for file in os.listdir(pos_dir)[0:]:
                #删除该文件夹里面的东西
                #先通过os.listdir获取该文件夹内的文件名列表，然后逐一删除
                file_path=pos_dir+'/'+file 
                os.remove(file_path) #删除该文件
    else: 
        os.mkdir(pos_dir)
    if os.path.exists(Cascade):
    #先判断该文件夹是否已经存在， 
        if os.path.getsize(Cascade):
            #如果存在，判断该文件夹里面是否有东西
            for file in os.listdir(Cascade)[0:]:
                #删除该文件夹里面的东西
                #先通过os.listdir获取该文件夹内的文件名列表，然后逐一删除
                file_path=Cascade+'/'+file 
                os.remove(file_path) #删除该文件
    else: 
        os.mkdir(Cascade)
mkdir1()
print("done,2")

done,2


3.  接下来，让我们填写一些基本参数吧

In [10]:
num_pos=int(input("多少张正样本："))
num_neg=int(input("多少张负样本（建议为正样本数量的3倍）："))
numPos=input("每级分类器训练时所用到的正样本数目（一般为正样本数目的80%）：")
numNeg=input("每级分类器训练时所用到的负样本数目(一般为numPos的2-3倍):")
numStages=input("训练分类器的级数，强分类器的个数(12-20):")
precalcValBufSize=input("缓存大小，用于存储预先计算的特征值(3000就行）：")
precalcIdxBufSize=input("缓存大小，用于存储预先计算的特征索引(3000就行):")
w=int(input("训练的正样本的宽度（建议10-100，否则会很慢）:"))
h=int(input("训练的正样本的高（建议与宽度相同）:"))
featureType=input("训练使用的特征类型(Haar，LBP和HOG):")

4.  接下来，拍一些照片（数量与上面的设定值相同）吧。

In [11]:
cap=cv2.VideoCapture(1)
read=cap.read
imshow=cv2.imshow
p=ord('p')
q=ord('q')
d=ord('d')
waitKey=cv2.waitKey
def take_pos_photo():
    pos_txt=open('pos.txt','w+')
    num_pos1=0
    print("正样本照片拍摄（尽可能物品占满镜头,按p进行拍摄，按q退出,d删除上一张）")
    while cap.isOpened() and num_pos1<num_pos:
        _,frame=read()
        imshow("pos_take",frame)
        if waitKey(1)&0xFF==p:
            frame=cv2.resize(frame,(w,h),interpolation=cv2.INTER_CUBIC)
            cv2.imwrite('pos/pos'+str(num_pos1)+'.jpg',frame)
            pos_txt.write('pos/pos'+str(num_pos1)+'.jpg 1 0 0 '+str(w)+' '+str(h)+'\n')
            num_pos1+=1
        elif waitKey(1)&0xFF==q:
            break
        elif waitKey(1)&0xFF==d:
            num_pos1-=1
            os.remove('pos/pos'+str(num_pos1)+'.jpg')
            pos_txt.close()
            file_old=open('pos.txt','r')
            lines = [i for i in file_old]
            del lines[-1]
            file_old.close()
            pos_txt=open('pos.txt','w+')
            pos_txt.write(''.join(lines))
    cv2.destroyAllWindows()
def take_neg_photo():
    neg_txt=open('neg.txt','w+')
    num_neg1=0
    print("负样本照片拍摄（尽可能物品占满镜头,按p进行拍摄，按q退出,d删除上一张）")
    while cap.isOpened() and num_neg1<num_neg:
        _,frame=read()
        imshow("neg_take",frame)
        if waitKey(1)&0xFF==p:
            frame=cv2.resize(frame,(w,h),interpolation=cv2.INTER_CUBIC)
            cv2.imwrite('neg/neg'+str(num_neg1)+'.jpg',frame)
            neg_txt.write('neg/neg'+str(num_neg1)+'.jpg 1 0 0 '+str(w)+' '+str(h)+'\n')
            num_neg1+=1
        elif waitKey(1)&0xFF==q:
            break
        elif waitKey(1)&0xFF==d:
            num_neg1-=1
            os.remove('neg/negp'+str(num_neg1)+'.jpg')
            neg_txt.close()
            file_old=open('neg.txt','r')
            lines = [i for i in file_old]
            del lines[-1]
            file_old.close()
            neg_txt=open('neg.txt','w+')
            neg_txt.write(''.join(lines))
    cv2.destroyAllWindows()
take_pos_photo()
take_neg_photo()
cap.release()

正样本照片拍摄（尽可能物品占满镜头,按p进行拍摄，按q退出,d删除上一张）
负样本照片拍摄（尽可能物品占满镜头,按p进行拍摄，按q退出,d删除上一张）


5.  开始训练模型吧

In [12]:
os.system("release/opencv_createsamples.exe  -info pos.txt -vec pos.vec -num "+str(num_pos)+" -w "+str(w)+" -h "+str(h))
os.system("release/opencv_traincascade.exe  -data TrainCascadeClassification \
    -vec pos.vec -bg neg.txt -numPos "+str(numPos)+" -numNeg "+str(numNeg)+" -numStages "\
        +str(numStages)+" -precalcValBufSize  "+str(precalcValBufSize)+"  -precalcIdxBufSize  "\
            +str(precalcIdxBufSize)+" -featureType "+str(featureType)+"  -w "+str(w)+" -h "+str(h))

1